In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

En este script realizare un modelo inicial base, el procesamiento de datos viene del script 'data/processed/processed_data_Vbaseline.pynb'.

In [2]:
# Carga de datos
df_ratings = pd.read_parquet('../../data/raw/data_model/dataset_process_trainV0.1.parquet')

In [3]:
# Movemos variable rating a predecir al final del dataset
rating=df_ratings.pop('rating')
df_ratings.insert(13,'rating',rating.values)

In [4]:
# Dividimos en X e y
X_ratings = df_ratings.iloc[:, :-1]
y_rating = df_ratings.iloc[:, 13]

Funciones para preparar los datos para la posterior fase del modelado

In [5]:
def extract_numerics_columns(X):
    numeric_columns = X.select_dtypes(include='number')
    return numeric_columns

In [6]:
def extract_categorical_columns(X):
    categorical_columns = X.select_dtypes(exclude='number')
    return categorical_columns

In [7]:
# Codificacion de variables numericas
def numeric_normalize(X):
    # Extraemos las columnas numéricas
    numeric_columns=extract_numerics_columns(X)

    # Creamos un MinMaxScaler y normalizamos las columnas numéricas
    normalizer = MinMaxScaler()
    normalized_numeric_columns = normalizer.fit_transform(numeric_columns)

    # Retorna ndarray 
    return normalized_numeric_columns

In [8]:
# Codificacion de variables categoricas
def categorical_encoder(X):
    # Extraemos las columnas categoricas
    categorical_columns=extract_categorical_columns(X)
    coded_categorical=[]
    # encoder de cada columna categorica
    for column in categorical_columns:
        encoded_column = pd.Categorical(X[column]).codes
        coded_categorical.append(encoded_column)

    # Convierte la lista de arrays categorical a 2D, para poder concatenar correctamente
    coded_categorical = np.column_stack(coded_categorical)
    return coded_categorical
    

In [9]:
# Concatenamos la codificacion de los valores numericos y categoricos
def concat_types(X):
    numeric=numeric_normalize(X)
    categorical=categorical_encoder(X)
    
    return np.concatenate((numeric,categorical), axis=1)

In [10]:
X_ratings=concat_types(X_ratings) # Concatenacion

Creacion del modelo (Baseline)

In [11]:
# Dividir en val y train
X_train, X_val, y_train, y_val = train_test_split(X_ratings, y_rating, test_size=0.2, stratify=y_rating, random_state=123)

In [12]:
# 1. Modelo
# Modelado del dataset, objetivo: predecir variable rating.

# Entrenamos el modelo de regresion lineal
def baseline_lr_ratings(X_train, y_train):
    reg_trained = LinearRegression().fit(X_train, y_train)
    return reg_trained

In [13]:
baseline_model=baseline_lr_ratings(X_train, y_train)# Llamada y entrenamos modelo

In [14]:
# Predecimos el ratings
def predict_baseline(X_val):
    prediction= baseline_model.predict(X_val)
    return prediction

In [15]:
y_pred = predict_baseline(X_val)# Llamada y retornamos la prediccion

In [16]:
# Analizamos error cuadratico medio del modelo
mean_squared_error(y_val, y_pred)

1.0609190132361324

El ECM es bastante alto, indicando que en promedio tenemos un error de 1 en nuestras valoraciones.
Al ser una baseline la usaremos para medir otros modelos

Prediccion con el conjunto de prueba

In [22]:
# Carga de datos
df_prueba = pd.read_parquet('../../data/raw/data_model/dataset_process_testV0.1.parquet')

In [23]:
y_true_prueba = df_prueba['rating']# Nos quedamos con la y real, para estimar el rendimiento del modelo ante datos no vistos

In [24]:
df_prueba.drop('rating', axis=1, inplace=True)# Eliminamos y 'rating'

In [25]:
X_prueba=concat_types(df_prueba)# procesamos los numericos y categoricos

In [27]:
y_pred_prueba = predict_baseline(X_prueba)# Llamada y retornamos la prediccion

In [28]:
# Analizamos error cuadratico medio del modelo
mean_squared_error(y_true_prueba, y_pred_prueba)

1.0756326423441425

El error en el conjunto de prueba es del 1 punto en promedio